In [0]:
gcs_bucket_path = "gs://nyc_trip_data_analysis/"
# List all files in the bucket
file_list = dbutils.fs.ls(gcs_bucket_path)

In [0]:
file_list

Out[3]: [FileInfo(path='gs://nyc_trip_data_analysis/shape_file.csv', name='shape_file.csv', size=3773723, modificationTime=1696445518184),
 FileInfo(path='gs://nyc_trip_data_analysis/taxi_zones.dbf', name='taxi_zones.dbf', size=147506, modificationTime=1695526324943),
 FileInfo(path='gs://nyc_trip_data_analysis/taxi_zones.prj', name='taxi_zones.prj', size=562, modificationTime=1695526335376),
 FileInfo(path='gs://nyc_trip_data_analysis/taxi_zones.sbn', name='taxi_zones.sbn', size=2908, modificationTime=1695526347177),
 FileInfo(path='gs://nyc_trip_data_analysis/taxi_zones.sbx', name='taxi_zones.sbx', size=348, modificationTime=1695526358668),
 FileInfo(path='gs://nyc_trip_data_analysis/taxi_zones.shp', name='taxi_zones.shp', size=1586264, modificationTime=1695526371288),
 FileInfo(path='gs://nyc_trip_data_analysis/taxi_zones.shx', name='taxi_zones.shx', size=2204, modificationTime=1695526382433),
 FileInfo(path='gs://nyc_trip_data_analysis/shape_file.xlsx', name='shape_file.xlsx', size

In [0]:
fhvhv_data = None
green_data = None
yellow_data = None
# Loop through the files and read them into the corresponding DataFrames
for file_info in file_list:
    file_path = file_info.path
    # Check if the file is in Parquet format and has "2022" in its name
    if file_path.endswith(".parquet") and (int(file_path.split('-')[0][-4:])) > 2015:
    # if file_path.endswith(".parquet") and "2022" in file_path:
        if "fhvhv_tripdata" in file_path:
            if fhvhv_data is None:
                fhvhv_data = spark.read.parquet(file_path)
            else:
                fhv_data = fhvhv_data.unionAll(spark.read.parquet(file_path))
        elif "green_tripdata" in file_path:
            if green_data is None:
                green_data = spark.read.parquet(file_path)
            else:
                green_data = green_data.unionAll(spark.read.parquet(file_path))
        elif "yellow_tripdata" in file_path:
            if yellow_data is None:
                yellow_data = spark.read.parquet(file_path)
            else:
                yellow_data = yellow_data.unionAll(spark.read.parquet(file_path))

# Now you have separate DataFrames for fhv, green, and yellow data from the year 2022


In [0]:
fhvhv_data.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [0]:
green_data.show()

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2016-01-01 00:17:40|  2016-01-01 00:36:15|                 N|       1.0|         255|         228|            1.0|         8.46|       25.5|  0.5|    0.

In [0]:
yellow_data.show()

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2016-01-01 00:12:22|  2016-01-01 00:29:14|            1.0|          3.2|       1.0|                 N|          48|         262|           1|       14.0|  0.5|    0.5|      3.0

In [0]:
from pyspark.sql.functions import to_timestamp

# Assuming 'yellow_2023Jan_trips' is your DataFrame
yellow_data = yellow_data.withColumn("tpep_pickup_datetime", to_timestamp("tpep_pickup_datetime"))
yellow_data = yellow_data.withColumn("tpep_dropoff_datetime", to_timestamp("tpep_dropoff_datetime"))
green_data = green_data.withColumn("lpep_pickup_datetime", to_timestamp("lpep_pickup_datetime"))
green_data = green_data.withColumn("lpep_dropoff_datetime", to_timestamp("lpep_dropoff_datetime"))
fhvhv_data = fhvhv_data.withColumn("pickup_datetime", to_timestamp("pickup_datetime"))
fhvhv_data = fhvhv_data.withColumn("pickup_datetime", to_timestamp("pickup_datetime"))


In [0]:
from pyspark.sql.functions import lit
yellow_data = yellow_data.withColumn("Taxi_Type", lit("Yellow"))
green_data = green_data.withColumn("Taxi_Type", lit("Green"))

In [0]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Define the mapping function
def map_license_to_taxi_type(license_num):
    if license_num == 'HV0002':
        return 'Juno'
    elif license_num == 'HV0003':
        return 'Uber'
    elif license_num == 'HV0004':
        return 'Via'
    elif license_num == 'HV0005':
        return 'Lyft'
    else:
        return 'Other'

# Register the mapping function as a UDF (User-Defined Function)
map_license_udf = udf(map_license_to_taxi_type, StringType())

# Create a new 'Taxi_Type' column based on 'hvfhs_license_num'
fhvhv_data = fhvhv_data.withColumn("Taxi_Type", map_license_udf(fhvhv_data['hvfhs_license_num']))


In [0]:
# Rename 'tpep_pickup_datetime' to 'pickup_datetime' in yellow_2023Jan_trips
yellow_data = yellow_data.withColumnRenamed('tpep_pickup_datetime', 'pickup_datetime')
yellow_data = yellow_data.withColumnRenamed('tpep_dropoff_datetime', 'dropoff_datetime')

# Rename 'lpep_pickup_datetime' to 'pickup_datetime' in green_2023Jan_trips
green_data = green_data.withColumnRenamed('lpep_pickup_datetime', 'pickup_datetime')
green_data = green_data.withColumnRenamed('lpep_dropoff_datetime', 'dropoff_datetime')

# Rename 'trip_miles' to 'trip_distance' in fhvhv_tripdata_2023Jan
fhvhv_data = fhvhv_data.withColumnRenamed('trip_miles', 'trip_distance')


In [0]:
# Read the CSV file with header
shape_file = spark.read.option("header", "true").csv("gs://nyc_trip_data_analysis/taxi_zone_lookup.csv")

In [0]:
shape_file.show(5)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+
only showing top 5 rows



In [0]:
from pyspark.sql.functions import col

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
yellow_data = yellow_data.join(shape_file.select('LocationID', 'Borough', 'Zone', 'service_zone'), yellow_data['PULocationID'] == shape_file['LocationID'], how='left')

# Drop the 'LocationID' column
yellow_data = yellow_data.drop('LocationID')

# Rename the columns
yellow_data = yellow_data.withColumnRenamed('Borough', 'PUBorough')
yellow_data = yellow_data.withColumnRenamed('Zone', 'PUZone')
yellow_data = yellow_data.withColumnRenamed('service_zone', 'PUservice_zone')


In [0]:
from pyspark.sql.functions import col

# Merge DataFrames based on 'DOLocationID' and 'LocationID' columns
yellow_data = yellow_data.join(shape_file.select('LocationID', 'Borough', 'Zone', 'service_zone'), yellow_data['DOLocationID'] == shape_file['LocationID'], how='left')

# Drop the 'LocationID' column
yellow_data = yellow_data.drop('LocationID')

# Rename the columns
yellow_data = yellow_data.withColumnRenamed('Borough', 'DOBorough')
yellow_data = yellow_data.withColumnRenamed('Zone', 'DOZone')
yellow_data = yellow_data.withColumnRenamed('service_zone', 'DOservice_zone')


In [0]:
from pyspark.sql.functions import col

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
green_data = green_data.join(shape_file.select('LocationID', 'Borough', 'Zone', 'service_zone'), green_data['PULocationID'] == shape_file['LocationID'], how='left')

# Drop the 'LocationID' column
green_data = green_data.drop('LocationID')

# Rename the columns
green_data = green_data.withColumnRenamed('Borough', 'PUBorough')
green_data = green_data.withColumnRenamed('Zone', 'PUZone')
green_data = green_data.withColumnRenamed('service_zone', 'PUservice_zone')


In [0]:
from pyspark.sql.functions import col

# Merge DataFrames based on 'DOLocationID' and 'LocationID' columns
green_data = green_data.join(shape_file.select('LocationID', 'Borough', 'Zone', 'service_zone'), green_data['DOLocationID'] == shape_file['LocationID'], how='left')

# Drop the 'LocationID' column
green_data = green_data.drop('LocationID')

# Rename the columns
green_data = green_data.withColumnRenamed('Borough', 'DOBorough')
green_data = green_data.withColumnRenamed('Zone', 'DOZone')
green_data = green_data.withColumnRenamed('service_zone', 'DOservice_zone')


In [0]:
from pyspark.sql.functions import col

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
fhvhv_data = fhvhv_data.join(shape_file.select('LocationID', 'Borough', 'Zone', 'service_zone'), fhvhv_data['PULocationID'] == shape_file['LocationID'], how='left')

# Drop the 'LocationID' column
fhvhv_data = fhvhv_data.drop('LocationID')

# Rename the columns
fhvhv_data = fhvhv_data.withColumnRenamed('Borough', 'PUBorough')
fhvhv_data = fhvhv_data.withColumnRenamed('Zone', 'PUZone')
fhvhv_data = fhvhv_data.withColumnRenamed('service_zone', 'PUservice_zone')


In [0]:
from pyspark.sql.functions import col

# Merge DataFrames based on 'DOLocationID' and 'LocationID' columns
fhvhv_data = fhvhv_data.join(shape_file.select('LocationID', 'Borough', 'Zone', 'service_zone'), fhvhv_data['DOLocationID'] == shape_file['LocationID'], how='left')

# Drop the 'LocationID' column
fhvhv_data = fhvhv_data.drop('LocationID')

# Rename the columns
fhvhv_data = fhvhv_data.withColumnRenamed('Borough', 'DOBorough')
fhvhv_data = fhvhv_data.withColumnRenamed('Zone', 'DOZone')
fhvhv_data = fhvhv_data.withColumnRenamed('service_zone', 'DOservice_zone')


In [0]:
!pip install geopandas

     |████████████████████████████████| 1.1 MB 4.1 MB/s 
     |█████                           | 2.4 

*** WARNING: max output size exceeded, skipping output. ***

     |████████████████████████████████| 8.3 MB 67.0 MB/s 
You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-ac09262b-2d95-4c50-82a5-9f276b7b54fe/bin/python -m pip install --upgrade pip' command.


In [0]:
import geopandas as gpd
# Read the shapefile using geopandas
gdf = gpd.read_file("gs://nyc_trip_data_analysis/taxi_zones.shp")

In [0]:
gdf['latitude'] = gdf['geometry'].apply(lambda geom: geom.centroid.y)
gdf['longitude'] = gdf['geometry'].apply(lambda geom: geom.centroid.x)

In [0]:
gdf.head()

,OBJECTID,Shape_Leng,Shape_Area,zone,LocationID,borough,geometry,latitude,longitude
0,1,0.116357,0.000782,Newark Airport,1,EWR,"POLYGON ((933100.918 192536.086, 933091.011 19...",191376.749531,9.359968e+05
1,2,0.433470,0.004866,Jamaica Bay,2,Queens,"MULTIPOLYGON (((1033269.244 172126.008, 103343...",164018.754403,1.031086e+06
2,3,0.084341,0.000314,Allerton/Pelham Gardens,3,Bronx,"POLYGON ((1026308.770 256767.698, 1026495.593 ...",254265.478659,1.026453e+06
3,4,0.043567,0.000112,Alphabet City,4,Manhattan,"POLYGON ((992073.467 203714.076, 992068.667 20...",202959.782391,9.906340e+05
4,5,0.092146,0.000498,Arden Heights,5,Staten Island,"POLYGON ((935843.310 144283.336, 936046.565 14...",140681.351376,9.318714e+05


In [0]:
shape = gdf[["LocationID","latitude", "longitude"]]
shape = spark.createDataFrame(shape)

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("ShapefileReader").getOrCreate()

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
fhvhv_data = fhvhv_data.join(shape.select('LocationID', 'latitude', 'longitude'), fhvhv_data['PULocationID'] == shape['LocationID'], how='left')

# Drop the 'LocationID' column
fhvhv_data = fhvhv_data.drop('LocationID')

# Rename the columns
fhvhv_data = fhvhv_data.withColumnRenamed('latitude', 'PUlatitude')
fhvhv_data = fhvhv_data.withColumnRenamed('longitude', 'PUlongitude')


In [0]:
from pyspark.sql.functions import col

# Assuming 'fhvhv_tripdata_2023Jan' is your DataFrame and 'shape_file' is another DataFrame

# Merge DataFrames based on 'DOLocationID' and 'LocationID' columns
fhvhv_data = fhvhv_data.join(shape.select('LocationID', 'latitude', 'longitude'), fhvhv_data['DOLocationID'] == shape['LocationID'], how='left')

# Drop the 'LocationID' column
fhvhv_data = fhvhv_data.drop('LocationID')

# Rename the columns
fhvhv_data = fhvhv_data.withColumnRenamed('latitude', 'DOlatitude')
fhvhv_data = fhvhv_data.withColumnRenamed('longitude', 'DOlongitude')


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("ShapefileReader").getOrCreate()

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
yellow_data = yellow_data.join(shape.select('LocationID', 'latitude', 'longitude'), yellow_data['PULocationID'] == shape['LocationID'], how='left')

# Drop the 'LocationID' column
yellow_data = yellow_data.drop('LocationID')

# Rename the columns
yellow_data = yellow_data.withColumnRenamed('latitude', 'PUlatitude')
yellow_data = yellow_data.withColumnRenamed('longitude', 'PUlongitude')


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("ShapefileReader").getOrCreate()

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
yellow_data = yellow_data.join(shape.select('LocationID', 'latitude', 'longitude'), yellow_data['DOLocationID'] == shape['LocationID'], how='left')

# Drop the 'LocationID' column
yellow_data = yellow_data.drop('LocationID')

# Rename the columns
yellow_data = yellow_data.withColumnRenamed('latitude', 'DOlatitude')
yellow_data = yellow_data.withColumnRenamed('longitude', 'DOlongitude')


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("ShapefileReader").getOrCreate()

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
green_data = green_data.join(shape.select('LocationID', 'latitude', 'longitude'), green_data['PULocationID'] == shape['LocationID'], how='left')

# Drop the 'LocationID' column
green_data = green_data.drop('LocationID')

# Rename the columns
green_data = green_data.withColumnRenamed('latitude', 'PUlatitude')
green_data = green_data.withColumnRenamed('longitude', 'PUlongitude')

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize a Spark session
spark = SparkSession.builder.appName("ShapefileReader").getOrCreate()

# Merge DataFrames based on 'PULocationID' and 'LocationID' columns
green_data = green_data.join(shape.select('LocationID', 'latitude', 'longitude'), green_data['DOLocationID'] == shape['LocationID'], how='left')

# Drop the 'LocationID' column
green_data = green_data.drop('LocationID')

# Rename the columns
green_data = green_data.withColumnRenamed('latitude', 'DOlatitude')
green_data = green_data.withColumnRenamed('longitude', 'DOlongitude')

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Initialize a Spark session
spark = SparkSession.builder.appName("DataframeManipulation").getOrCreate()

# Assuming 'yellow_2023Jan_trips', 'green_2023Jan_trips', and 'fhvhv_tripdata_2023Jan' are your DataFrames
dataframes = [yellow_data, green_data, fhvhv_data]

# Define a common column list
common_columns = ['pickup_datetime', 'dropoff_datetime', 'trip_distance', 'Taxi_Type', 'PULocationID', 'DOLocationID', 'PUlatitude', 'PUlongitude', 'DOlatitude', 'DOlongitude', 'PUBorough', 'PUZone', 'PUservice_zone', 'DOBorough', 'DOZone', 'DOservice_zone']

# Initialize an empty list to store modified DataFrames
modified_dataframes = []

# Initialize a trip_id counter
trip_id_counter = 1

# Iterate through each DataFrame
for df in dataframes:
    # Extract the common columns
    common_df = df.select(common_columns)
    
    # Add a 'trip_id' column with unique values for each DataFrame
    common_df = common_df.withColumn('trip_id', monotonically_increasing_id() + trip_id_counter)
    
    # Append the modified DataFrame to the list
    modified_dataframes.append(common_df)
    
    # Update the trip_id counter
    trip_id_counter += common_df.count()

# Union all the modified DataFrames into a single DataFrame
from functools import reduce
from pyspark.sql import DataFrame

combined_df = reduce(DataFrame.unionAll, modified_dataframes)


In [0]:
# Move 'trip_id' to the first position in the DataFrame
combined_df = combined_df.select('trip_id', *common_columns)

In [0]:
combined_df.show(5)

+-------+-------------------+-------------------+-------------+---------+------------+------------+------------------+-----------------+------------------+-----------------+---------+--------------------+--------------+---------+---------------+--------------+
|trip_id|    pickup_datetime|   dropoff_datetime|trip_distance|Taxi_Type|PULocationID|DOLocationID|        PUlatitude|      PUlongitude|        DOlatitude|      DOlongitude|PUBorough|              PUZone|PUservice_zone|DOBorough|         DOZone|DOservice_zone|
+-------+-------------------+-------------------+-------------+---------+------------+------------+------------------+-----------------+------------------+-----------------+---------+--------------------+--------------+---------+---------------+--------------+
|      1|2016-01-01 00:12:22|2016-01-01 00:29:14|          3.2|   Yellow|          48|         262|216986.07003374878|987063.1857021254|  221974.400787808|999064.7250824339|Manhattan|        Clinton East|   Yellow Zon

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Initialize a Spark session
spark = SparkSession.builder.appName("FilterDataFrame").getOrCreate()

# Assuming you have a DataFrame named 'yellow_data'

# Define the conditions for RateCodeID and Payment_type
RateCodeName = [
    (col("RateCodeID") == 1, "Standard rate"),
    (col("RateCodeID") == 2, "JFK"),
    (col("RateCodeID") == 3, "Newark"),
    (col("RateCodeID") == 4, "Nassau or Westchester"),
    (col("RateCodeID") == 5, "Negotiated fare"),
    (col("RateCodeID") == 6, "Group ride")
]

PaymentTypeName = [
    (col("Payment_type") == 1, "Credit card"),
    (col("Payment_type") == 2, "Cash"),
    (col("Payment_type") == 3, "No charge"),
    (col("Payment_type") == 4, "Dispute"),
    (col("Payment_type") == 5, "Unknown"),
    (col("Payment_type") == 6, "Voided trip")
]

# Apply the conditions using 'when' and 'otherwise' functions
yellow_data = yellow_data.withColumn("RateCodeName", when(RateCodeName[0][0], RateCodeName[0][1])
                            .when(RateCodeName[1][0], RateCodeName[1][1])
                            .when(RateCodeName[2][0], RateCodeName[2][1])
                            .when(RateCodeName[3][0], RateCodeName[3][1])
                            .when(RateCodeName[4][0], RateCodeName[4][1])
                            .when(RateCodeName[5][0], RateCodeName[5][1])
                            .otherwise("Other"))

yellow_data = yellow_data.withColumn("PaymentTypeName", when(PaymentTypeName[0][0], PaymentTypeName[0][1])
                            .when(PaymentTypeName[1][0], PaymentTypeName[1][1])
                            .when(PaymentTypeName[2][0], PaymentTypeName[2][1])
                            .when(PaymentTypeName[3][0], PaymentTypeName[3][1])
                            .when(PaymentTypeName[4][0], PaymentTypeName[4][1])
                            .when(PaymentTypeName[5][0], PaymentTypeName[5][1])
                            .otherwise("Other"))


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Initialize a Spark session
spark = SparkSession.builder.appName("FilterDataFrame").getOrCreate()

# Assuming you have a DataFrame named 'yellow_data'

# Define the conditions for RateCodeID and Payment_type
RateCodeName = [
    (col("RateCodeID") == 1, "Standard rate"),
    (col("RateCodeID") == 2, "JFK"),
    (col("RateCodeID") == 3, "Newark"),
    (col("RateCodeID") == 4, "Nassau or Westchester"),
    (col("RateCodeID") == 5, "Negotiated fare"),
    (col("RateCodeID") == 6, "Group ride")
]

PaymentTypeName = [
    (col("Payment_type") == 1, "Credit card"),
    (col("Payment_type") == 2, "Cash"),
    (col("Payment_type") == 3, "No charge"),
    (col("Payment_type") == 4, "Dispute"),
    (col("Payment_type") == 5, "Unknown"),
    (col("Payment_type") == 6, "Voided trip")
]

# Apply the conditions using 'when' and 'otherwise' functions
yellow_data = yellow_data.withColumn("RateCodeName", when(RateCodeName[0][0], RateCodeName[0][1])
                            .when(RateCodeName[1][0], RateCodeName[1][1])
                            .when(RateCodeName[2][0], RateCodeName[2][1])
                            .when(RateCodeName[3][0], RateCodeName[3][1])
                            .when(RateCodeName[4][0], RateCodeName[4][1])
                            .when(RateCodeName[5][0], RateCodeName[5][1])
                            .otherwise("Other"))

yellow_data = yellow_data.withColumn("PaymentTypeName", when(PaymentTypeName[0][0], PaymentTypeName[0][1])
                            .when(PaymentTypeName[1][0], PaymentTypeName[1][1])
                            .when(PaymentTypeName[2][0], PaymentTypeName[2][1])
                            .when(PaymentTypeName[3][0], PaymentTypeName[3][1])
                            .when(PaymentTypeName[4][0], PaymentTypeName[4][1])
                            .when(PaymentTypeName[5][0], PaymentTypeName[5][1])
                            .otherwise("Other"))


In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Initialize a Spark session
spark = SparkSession.builder.appName("FilterDataFrame").getOrCreate()

# Assuming you have a DataFrame named 'yellow_data'

# Define the conditions for RateCodeID and Payment_type
VendorName = [
    (col("VendorID") == 1, " Creative Mobile Technologies, LLC"),
    (col("VendorID") == 2, " VeriFone Inc.")
]

# Apply the conditions using 'when' and 'otherwise' functions
yellow_data = yellow_data.withColumn("VendorName", when(VendorName[0][0], VendorName[0][1])
                            .when(VendorName[1][0], VendorName[1][1])
                            .otherwise("Other"))

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, col

# Initialize a Spark session
spark = SparkSession.builder.appName("FilterDataFrame").getOrCreate()

# Define the conditions for RateCodeID and Payment_type
RateCodeName = [
    (col("RateCodeID") == 1, "Standard rate"),
    (col("RateCodeID") == 2, "JFK"),
    (col("RateCodeID") == 3, "Newark"),
    (col("RateCodeID") == 4, "Nassau or Westchester"),
    (col("RateCodeID") == 5, "Negotiated fare"),
    (col("RateCodeID") == 6, "Group ride")
]

PaymentTypeName = [
    (col("Payment_type") == 1, "Credit card"),
    (col("Payment_type") == 2, "Cash"),
    (col("Payment_type") == 3, "No charge"),
    (col("Payment_type") == 4, "Dispute"),
    (col("Payment_type") == 5, "Unknown"),
    (col("Payment_type") == 6, "Voided trip")
]

VendorName = [
    (col("VendorID") == 1, " Creative Mobile Technologies, LLC"),
    (col("VendorID") == 2, " VeriFone Inc.")
]

# Apply the conditions using 'when' and 'otherwise' functions
green_data = green_data.withColumn("RateCodeName", when(RateCodeName[0][0], RateCodeName[0][1])
                            .when(RateCodeName[1][0], RateCodeName[1][1])
                            .when(RateCodeName[2][0], RateCodeName[2][1])
                            .when(RateCodeName[3][0], RateCodeName[3][1])
                            .when(RateCodeName[4][0], RateCodeName[4][1])
                            .when(RateCodeName[5][0], RateCodeName[5][1])
                            .otherwise("Other"))

green_data = green_data.withColumn("PaymentTypeName", when(PaymentTypeName[0][0], PaymentTypeName[0][1])
                            .when(PaymentTypeName[1][0], PaymentTypeName[1][1])
                            .when(PaymentTypeName[2][0], PaymentTypeName[2][1])
                            .when(PaymentTypeName[3][0], PaymentTypeName[3][1])
                            .when(PaymentTypeName[4][0], PaymentTypeName[4][1])
                            .when(PaymentTypeName[5][0], PaymentTypeName[5][1])
                            .otherwise("Other"))

green_data = green_data.withColumn("VendorName", when(VendorName[0][0], VendorName[0][1])
                            .when(VendorName[1][0], VendorName[1][1])
                            .otherwise("Other"))


In [0]:
yellow_data.count()

Out[38]: 547011375

In [0]:
table = "bigquery-public-data.samples.shakespeare"
df = spark.read.format("bigquery").option("table",table).load()
df.createOrReplaceTempView("shakespeare")

In [0]:
bucket = "databricksbigquery"
table = "nyctripdataset.yellowtaxi"

yellow_data.write.format("bigquery").option("temporaryGcsBucket", bucket).option("table", table).mode("overwrite").save()

In [0]:
bucket = "databricksbigquery"
table = "nyctripdataset.greentaxi"

green_data.write.format("bigquery").option("temporaryGcsBucket", bucket).option("table", table).mode("overwrite").save()

In [0]:
bucket = "databricksbigquery"
table = "nyctripdataset.fhvhvtaxi"

fhvhv_data.write.format("bigquery").option("temporaryGcsBucket", bucket).option("table", table).mode("overwrite").save()

In [0]:
bucket = "databricksbigquery"
table = "nyctripdataset.combinedtaxi"

combined_df.write.format("bigquery").option("temporaryGcsBucket", bucket).option("table", table).mode("overwrite").save()